# Importar

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# Datos

In [2]:
# Cargar el dataset preparado
df = pd.read_csv('../Data/dataset_preparado.csv')

# Preparación del modelo

In [3]:
# Separar las características (X) y la variable objetivo (y)
X = df.drop('deposit', axis=1)
y = df['deposit']

In [4]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Definir imputador para características numéricas
num_imputer = SimpleImputer(strategy='mean')

In [7]:
# Imputar características numéricas
X_train_numeric_imputed = num_imputer.fit_transform(X_train.select_dtypes(include=['int', 'float']))
X_test_numeric_imputed = num_imputer.transform(X_test.select_dtypes(include=['int', 'float']))

In [8]:
# Definir imputador para características categóricas
cat_imputer = SimpleImputer(strategy='most_frequent')

In [9]:
# Imputar características categóricas
X_train_categorical_imputed = cat_imputer.fit_transform(X_train.select_dtypes(include=['object']))
X_test_categorical_imputed = cat_imputer.transform(X_test.select_dtypes(include=['object']))

In [11]:
# Concatenar las características numéricas y categóricas imputadas
X_train_imputed = np.concatenate((X_train_numeric_imputed, X_train_categorical_imputed), axis=1)
X_test_imputed = np.concatenate((X_test_numeric_imputed, X_test_categorical_imputed), axis=1)

In [17]:
# Codificar las variables categóricas usando OneHotEncoding
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train.select_dtypes(include=['object']))
X_test_encoded = encoder.transform(X_test.select_dtypes(include=['object']))

In [58]:
from sklearn.preprocessing import OneHotEncoder

# Separar las características categóricas y numéricas
X_numeric = X.select_dtypes(include=['int', 'float'])
X_categorical = X.select_dtypes(include=['object'])

# Imputar las características categóricas
cat_imputer = SimpleImputer(strategy='most_frequent')
X_categorical_imputed = cat_imputer.fit_transform(X_categorical)

# Codificar las variables categóricas usando OneHotEncoding
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
X_categorical_encoded = encoder.fit_transform(X_categorical_imputed)

# Imputar las características numéricas
num_imputer = SimpleImputer(strategy='mean')
X_numeric_imputed = num_imputer.fit_transform(X_numeric)

# Concatenar las características codificadas con las características numéricas imputadas
X_imputed = np.concatenate((X_numeric_imputed, X_categorical_encoded), axis=1)

# Dividir el conjunto de datos imputado en conjuntos de entrenamiento y prueba
X_train_imputed, X_test_imputed, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)


In [26]:
from sklearn.metrics import accuracy_score, classification_report

# Entrenar el modelo RandomForestClassifier con los datos imputados y codificados
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_imputed, y_train)

# Predecir en el conjunto de prueba
y_pred = clf.predict(X_test_imputed)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Mostrar el reporte de clasificación
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)



Accuracy: 0.8410210479175997
Classification Report:
              precision    recall  f1-score   support

          no       0.87      0.82      0.84      1166
         yes       0.81      0.86      0.84      1067

    accuracy                           0.84      2233
   macro avg       0.84      0.84      0.84      2233
weighted avg       0.84      0.84      0.84      2233



# Preparación del modelo

In [27]:
feature_importances = clf.feature_importances_

In [29]:
feature_importances = clf.feature_importances_
#print(feature_importances)

# Optimización del Modelo

In [31]:
# Definir la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


In [32]:
# Instanciar el modelo con GridSearchCV
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')

In [33]:
# Ajustar el modelo
grid_search.fit(X_train_imputed, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [34]:
# Obtener el mejor estimador
best_clf = grid_search.best_estimator_

In [35]:
# Mostrar los mejores parámetros
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}


In [36]:
# Evaluar el modelo con los mejores parámetros
y_pred_best = best_clf.predict(X_test_imputed)
accuracy_best = accuracy_score(y_test, y_pred_best)
print("Best Accuracy:", accuracy_best)

Best Accuracy: 0.8410210479175997


# Validación Cruzada

In [38]:
# Realizar validación cruzada
cv_scores = cross_val_score(clf, X_imputed, y, cv=5)

In [39]:
# Mostrar los puntajes de validación cruzada
print("Cross Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

Cross Validation Scores: [0.72816838 0.75459024 0.81810036 0.86648746 0.84498208]
Mean CV Accuracy: 0.8024657026325548


# Interpreación

In [40]:
# Realizar validación cruzada
cv_scores = cross_val_score(clf, X_imputed, y, cv=5)

In [41]:
# Calcular la precisión media de la validación cruzada
mean_cv_accuracy = cv_scores.mean()

In [56]:
# Crear una matriz de decisiones para determinar el comentario
if mean_cv_accuracy >= 0.85:
    if all(score >= 0.85 for score in cv_scores):
        comentario = f"Al tener una precisión media de {mean_cv_accuracy:.2f} y puntajes de validación cruzada consistentemente altos, el rendimiento del modelo es excelente. Se puede confiar en él para predecir la suscripción de depósitos a plazo."
    else:
        comentario = f"A pesar de tener una precisión media de {mean_cv_accuracy:.2f}, algunos pliegues tienen puntajes de validación cruzada más bajos. Aunque el rendimiento general es bueno, se recomienda revisar esos pliegues y considerar posibles mejoras en esas áreas."
elif 0.75 <= mean_cv_accuracy < 0.85:
    if all(score >= 0.75 for score in cv_scores):
        comentario = f"Con una precisión media de {mean_cv_accuracy:.2f} y puntajes de validación cruzada aceptables, el rendimiento del modelo es aceptable. Aunque no es excelente, sigue siendo útil para predecir la suscripción de depósitos a plazo."
    else:
        comentario = f"Aunque la precisión media es de {mean_cv_accuracy:.2f}, algunos pliegues tienen puntajes de validación cruzada por debajo del umbral de aceptabilidad. Se recomienda revisar esos pliegues específicos y considerar estrategias para mejorar el rendimiento en esas áreas."
else:
    comentario = f"El rendimiento del modelo es insatisfactorio, con una precisión media de {mean_cv_accuracy:.2f}. Se requieren mejoras significativas para hacer predicciones confiables sobre la suscripción de depósitos a plazo. Se recomienda una revisión exhaustiva del modelo y una posible inclusión de más datos o características relevantes."


In [57]:
# Mostrar el comentario
print("\n\033[1mCOMENTARIO SOBRE EL ESTADO DEL MODELO:\033[0m")
print(comentario.upper())


COMENTARIO SOBRE EL ESTADO DEL MODELO:
AUNQUE LA PRECISIÓN MEDIA ES DE 0.80, ALGUNOS PLIEGUES TIENEN PUNTAJES DE VALIDACIÓN CRUZADA POR DEBAJO DEL UMBRAL DE ACEPTABILIDAD. SE RECOMIENDA REVISAR ESOS PLIEGUES ESPECÍFICOS Y CONSIDERAR ESTRATEGIAS PARA MEJORAR EL RENDIMIENTO EN ESAS ÁREAS.
